<a href="https://colab.research.google.com/github/dc250601/CMS-GSoC-2022/blob/main/Colab/Quark_gluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.2
Running on TPU  ['10.15.92.234:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.15.92.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.15.92.234:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [2]:
import re
import numpy as np
from matplotlib import pyplot as plt
import os
import numpy as np
import sys
sys.path.append('./eff_feature.py')
sys.path.append('./swin_sp.py')
#Modified efficient net to match the config of the timm 
import eff_feature as efm
#import swin_r as swin
import swin_sp as swin
AUTO = tf.data.experimental.AUTOTUNE

In [3]:
BATCH_SIZE = 128 * tpu_strategy.num_replicas_in_sync

In [4]:
GCS_DS_PATH = "gs://quark_gluon/Qg"
TRAIN = tf.io.gfile.glob(GCS_DS_PATH + '/Train/*.tfrecords')
TEST =  tf.io.gfile.glob(GCS_DS_PATH + '/Test/*.tfrecords')

In [5]:
def _parse_image_label_function(example_proto):
    # Create a dictionary describing the features.
    image_feature_description = {
        'raw_image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'name': tf.io.FixedLenFeature([], tf.string)
        
    }
    # Parse the input tf.train.Example proto using the dictionary above.
    content = tf.io.parse_single_example(example_proto, image_feature_description)
    content["raw_image"] = tf.io.decode_png(content["raw_image"], channels = 3)
    content["raw_image"] = tf.cast(content["raw_image"], tf.float32) / 255.0
#    content["raw_image"] = tf.reshape(["raw_image"], [224,224, 3])
#     image = content["raw_image"]
#     label = content["label"]
    return content["raw_image"],content["label"]


def data_augment(image, label):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    image = tf.image.resize(image, (224,224))
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label  



def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_image_label_function, num_parallel_calls=AUTO)
    dataset = dataset.map(data_augment,num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
    dataset = dataset.prefetch(AUTO)
    
    return dataset

In [6]:
train_ds = load_dataset(TRAIN)
val_ds = load_dataset(TEST)

In [7]:
with tpu_strategy.scope():

    eff_blocks = 2
    
    model_a = swin.SwinTransformer(part_model  =True, eff_blocks = eff_blocks, partition_ = 1)

    model_b = swin.SwinTransformer(part_model  =True, eff_blocks = eff_blocks, partition_ = 2)

    model_ef = efm.EfficientNetB3(input_shape=(224,224,3),feature_extractor_blocks=eff_blocks)
    
    model_top = tf.keras.layers.Dense(1, activation = "sigmoid")

    feature_extractor_blocks=eff_blocks
    swin_blocks = model_b.num_layers - feature_extractor_blocks
    embeded_dim = model_a.embed_dim[0] * (2**(4 - swin_blocks))
    connector = tf.keras.Sequential([tf.keras.layers.Conv2D(filters=embeded_dim,use_bias=False, kernel_size = 1),
                                     tf.keras.layers.BatchNormalization(momentum = 0.1, epsilon = 1e-05),
                                     tf.keras.layers.Activation("swish"),
                                     tf.keras.layers.Reshape((-1,embeded_dim))
    ])

    class Stage1(tf.keras.Model):

        def __init__(self, model_a, model_b, top):
            super(Stage1,self).__init__()
            self.model_a = model_a
            self.model_b = model_b
            self.top = top
            

        def call(self, x):
            x = self.model_a(x)
            x = self.model_b(x)
            x = self.top(x)
            return x


    class Stage2(tf.keras.Model):

        def __init__(self, eff, connector, model_b, top):
            super().__init__()
            self.eff = eff
            self.connector = connector
            self.model_b = model_b
            self.model_b.trainable = False
            self.top = top
        def call(self, x):
            x = self.eff(x)
            x = self.connector(x)
            x = self.model_b(x)
            x = self.top(x)
            return x


    class Stage3(tf.keras.Model):

        def __init__(self, eff, connector, model_b, top):
            super().__init__()
            self.eff = eff
            self.connector = connector
            self.model_b = model_b
            self.model_b.trainable = True
            self.top = top
        def call(self, x):
            x = self.eff(x)
            x = self.connector(x)
            x = self.model_b(x)
            x = self.top(x)
            return x

In [8]:
with tpu_strategy.scope():
    model = swin.SwinTransformer(num_classes=1, pre_logits = True, drop_path_rate=0.1)
    ld = model(tf.random.uniform((1,224,224,3)), training = False)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape = (224,224,3)),
        model,
        tf.keras.layers.Dense(1, activation = "sigmoid")
        
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr = 0.0001),
        # steps_per_execution = 4,
        loss = tf.keras.losses.BinaryCrossentropy(),
        
        metrics=["accuracy", "AUC"]
    )

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
history = model.fit(train_ds,
#                     steps_per_epoch= 1,
                    steps_per_epoch=194*2,
                    validation_data=val_ds,
                    validation_steps=34*2,
                    epochs=10
                    )

Epoch 1/10
388/388 [==============================] - 543s 1s/step - loss: 0.7028 - accuracy: 0.6040 - auc: 0.6510 - val_loss: 0.5980 - val_accuracy: 0.6835 - val_auc: 0.7635
Epoch 2/10
388/388 [==============================] - 448s 1s/step - loss: 0.5714 - accuracy: 0.7095 - auc: 0.7740 - val_loss: 0.5674 - val_accuracy: 0.7113 - val_auc: 0.7779
Epoch 3/10
 14/388 [>.............................] - ETA: 6:49 - loss: 0.5685 - accuracy: 0.7137 - auc: 0.7774